# ISBN 10

In [21]:
def calculateCheckDigit(isbn10):
    sum = 0
    for i in range(9):
        sum += (i + 1) * int(isbn10[i])
    checkDigit = sum % 11
    if checkDigit == 10:
        return 'X'
    else:
        return str(checkDigit)

In [22]:
isbn = ["305501517", "047182819", "426379215"]

for i in isbn:
    print(f"isbn10: {i}{calculateCheckDigit(i)}")

isbn10: 3055015177
isbn10: 047182819X
isbn10: 4263792157
